In [ ]:
pip install opencv-python keras tensorflow matplotlib

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Define the paths to the dataset
train_data_dir = 'C:/Users/Admin/Desktop/reaction_pre-train_dataset/test'  # Replace with the actual path
test_data_dir = 'C:/Users/Admin/Desktop/reaction_pre-train_dataset/train'  # Replace with the actual path

# Define image dimensions and batch size
img_size = 48
batch_size = 64
num_classes = 7  # 7 emotions: Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral

# Function to load data
def load_data(data_dir):
    categories = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    data = []
    labels = []
    
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)  # Assigning labels for each category
        
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_img = cv2.resize(img_array, (img_size, img_size))
                data.append(resized_img)
                labels.append(class_num)
            except Exception as e:
                print(f"Error loading image {img}: {e}")
    
    data = np.array(data).reshape(-1, img_size, img_size, 1)  # Reshaping to fit the model
    labels = np.array(labels)
    
    return data, labels

# Load the training and testing data
train_data, train_labels = load_data(train_data_dir)
test_data, test_labels = load_data(test_data_dir)

# Normalize the pixel values to range [0, 1]
train_data = train_data / 255.0
test_data = test_data / 255.0

# One-hot encode the labels
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

# Model architecture
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_size, img_size, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Train the model
history = model.fit(
    datagen.flow(train_data, train_labels, batch_size=batch_size),
    epochs=25,
    validation_data=(test_data, test_labels),
    steps_per_epoch=len(train_data) // batch_size
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Save the model
model.save('emotion_detection_model.h5')

# Classification report
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(test_labels, axis=1)

print(classification_report(true_labels, predicted_labels, target_names=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']))

# Confusion matrix
print(confusion_matrix(true_labels, predicted_labels))

# Function to predict emotion from a new image
def predict_emotion(image_path):
    categories = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_size, img_size))
    img = img.reshape(1, img_size, img_size, 1)
    img = img / 255.0
    
    prediction = model.predict(img)
    max_index = np.argmax(prediction)
    emotion = categories[max_index]
    
    print(f"Predicted Emotion: {emotion}")

# Example usage
predict_emotion('C:/Users/Admin/Desktop/Kids/4be1baf30d5f4c9132025ff7697dbc5b.jpg')  # Replace with the actual path

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Define the paths to the dataset
train_data_dir = 'C:/Users/Admin/Desktop/reaction_pre-train_dataset/train'  # Replace with the actual path
test_data_dir = 'C:/Users/Admin/Desktop/reaction_pre-train_dataset/test'    # Replace with the actual path

# Define image dimensions and batch size
img_size = 48
batch_size = 64
num_classes = 7  # 7 emotions: Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load data using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_size, img_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical"
)

# Model architecture
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_size, img_size, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=test_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator)
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Save the model
model.save('emotion_detection_model.h5')

# Classification report
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_generator.classes

print(classification_report(true_labels, predicted_labels, target_names=train_generator.class_indices.keys()))

# Confusion matrix
print(confusion_matrix(true_labels, predicted_labels))

# Function to predict emotion from a new image
def predict_emotion(image_path):
    categories = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    
    img = tf.keras.preprocessing.image.load_img(image_path, color_mode="grayscale", target_size=(img_size, img_size))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img.reshape(1, img_size, img_size, 1)
    img = img / 255.0
    
    prediction = model.predict(img)
    max_index = np.argmax(prediction)
    emotion = categories[max_index]
    
    print(f"Predicted Emotion: {emotion}")

# Example usage
predict_emotion('C:/Users/Admin/Desktop/Kids/4be1baf30d5f4c9132025ff7697dbc5b.jpg')  # Replace with the actual path

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Define the paths to the dataset
train_data_dir = 'C:/Users/Admin/Desktop/reaction_pre-train_dataset/train'  # Replace with the actual path
test_data_dir = 'C:/Users/Admin/Desktop/reaction_pre-train_dataset/test'    # Replace with the actual path

# Define image dimensions and batch size
img_size = 48
batch_size = 64
num_classes = 7  # 7 emotions: Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load data using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_size, img_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical"
)

# Model architecture
model = Sequential()

# Use an explicit Input layer for better control
model.add(Input(shape=(img_size, img_size, 1)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Steps for training and validation
steps_per_epoch = train_generator.samples // batch_size
validation_steps = test_generator.samples // batch_size

# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=test_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Save the model
model.save('emotion_detection_model.h5')

# Classification report and confusion matrix
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_generator.classes

print(classification_report(true_labels, predicted_labels, target_names=list(train_generator.class_indices.keys())))
print(confusion_matrix(true_labels, predicted_labels))

# Function to predict emotion from a new image
def predict_emotion(image_path):
    categories = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    
    img = tf.keras.preprocessing.image.load_img(image_path, color_mode="grayscale", target_size=(img_size, img_size))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img.reshape(1, img_size, img_size, 1)
    img = img / 255.0
    
    prediction = model.predict(img)
    max_index = np.argmax(prediction)
    emotion = categories[max_index]
    
    print(f"Predicted Emotion: {emotion}")

# Example usage
predict_emotion('C:/Users/Admin/Desktop/Kids/4be1baf30d5f4c9132025ff7697dbc5b.jpg')  # Replace with the actual path

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

# Load the trained model
model = tf.keras.models.load_model('emotion_detection_model.h5')

# Define image dimensions and categories
img_size = 48
categories = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Function to predict emotion from a new image
def predict_emotion(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, color_mode="grayscale", target_size=(img_size, img_size))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img.reshape(1, img_size, img_size, 1)
    img = img / 255.0
    
    prediction = model.predict(img)
    max_index = np.argmax(prediction)
    emotion = categories[max_index]
    
    print(f"Predicted Emotion: {emotion}")

# Example usage for a new image
image_path = 'C:/Users/Admin/Desktop/Kids/istockphoto-515683938-612x612.jpg'  # Replace with the actual path of the new image
predict_emotion(image_path)

In [ ]:
import numpy as np
import tensorflow as tf
import cv2  # OpenCV library for image processing

# Load the trained model
model = tf.keras.models.load_model('emotion_detection_model.h5')

# Define image dimensions and categories
img_size = 48
categories = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Function to predict emotion from a new image and display it with a green frame and text
def predict_emotion(image_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    
    # Resize the image for the model prediction (48x48 grayscale)
    face = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    face_resized = cv2.resize(face, (img_size, img_size))  # Resize the image to 48x48
    face_resized = face_resized.reshape(1, img_size, img_size, 1)
    face_resized = face_resized / 255.0  # Normalize the image

    # Make a prediction
    prediction = model.predict(face_resized)
    max_index = np.argmax(prediction)
    emotion = categories[max_index]
    
    # Display the emotion on the image and draw a green frame
    color = (0, 255, 0)  # Green color for the frame and text
    thickness = 2  # Thickness of the frame

    # Draw a green rectangle around the image (simulating a frame)
    h, w, _ = img.shape
    cv2.rectangle(img, (10, 10), (w-10, h-10), color, thickness)

    # Add the predicted emotion text on top of the image
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, emotion, (20, 50), font, 1, color, 2, cv2.LINE_AA)

    # Display the resulting image with the green frame and emotion text
    cv2.imshow("Emotion Prediction", img)
    
    # Wait for a key press and close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage for a new image
image_path = 'C:/Users/Admin/Desktop/Kids/05-12-21-happy-people.jpg'  # Replace with the actual path of the new image
predict_emotion(image_path)


In [ ]:
import numpy as np
import tensorflow as tf
import cv2  # OpenCV library for image processing

# Load the trained model
model = tf.keras.models.load_model('emotion_detection_model.h5')

# Define image dimensions and categories
img_size = 48
categories = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Function to predict emotion from a new image and save the output
def predict_emotion(image_path, save_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    
    # Resize the image for the model prediction (48x48 grayscale)
    face = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    face_resized = cv2.resize(face, (img_size, img_size))  # Resize the image to 48x48
    face_resized = face_resized.reshape(1, img_size, img_size, 1)
    face_resized = face_resized / 255.0  # Normalize the image

    # Make a prediction
    prediction = model.predict(face_resized)
    max_index = np.argmax(prediction)
    emotion = categories[max_index]
    
    # Display the emotion on the image and draw a green frame
    color = (0, 255, 0)  # Green color for the frame and text
    thickness = 2  # Thickness of the frame

    # Draw a green rectangle around the image (simulating a frame)
    h, w, _ = img.shape
    cv2.rectangle(img, (10, 10), (w-10, h-10), color, thickness)

    # Add the predicted emotion text on top of the image
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, emotion, (20, 50), font, 1, color, 2, cv2.LINE_AA)

    # Save the resulting image with the green frame and emotion text
    cv2.imwrite(save_path, img)

    # Optionally, display the image (you can comment this out if you just want to save)
    cv2.imshow("Emotion Prediction", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage for a new image
image_path = 'C:/Users/Admin/Desktop/Kids/05-12-21-happy-people.jpg'  # Input image path
save_path = 'C:/Users/Admin/Desktop/Kids/predicted_emotion_output.jpg'  # Output image save path
predict_emotion(image_path, save_path)

In [ ]:
import numpy as np
import cv2
import tensorflow as tf

# Preprocess function for grayscale images
def preprocess_image(image_path):
    img_size = 48  # Assuming the model was trained on 48x48 images
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image in grayscale
    img_resized = cv2.resize(img, (img_size, img_size))  # Resize to 48x48
    img_normalized = img_resized / 255.0  # Normalize pixel values (0-1)
    img_reshaped = img_normalized.reshape(1, img_size, img_size, 1)  # Reshape for the model (1 channel for grayscale)
    return img_reshaped

# Paths to the two images
image_paths = [
    r'C:\Users\Admin\Desktop\Kids\05-12-21-happy-people.jpg', 
    r'C:\Users\Admin\Desktop\Kids\360_F_73243993_WfQ6CwwVbsSf36W0oLaCseTecG6dQMFm.jpg'
]

# Load the trained model
model = tf.keras.models.load_model('emotion_detection_model.h5')

# Define the emotion categories (ensure these match your model's categories)
categories = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Loop over each image, preprocess, and predict
for image_path in image_paths:
    image = preprocess_image(image_path)
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction, axis=1)
    
    # Print the predicted emotion
    print(f"Image: {image_path} - Predicted Emotion: {categories[predicted_class[0]]}")

In [1]:
import numpy as np
import tensorflow as tf
import cv2  # OpenCV library for image processing
import os

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Load the trained model
model = tf.keras.models.load_model('emotion_detection_model.h5')

# Define image dimensions and categories
img_size = 48
categories = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Function to predict emotion from a new image and display it with a green frame and text
def predict_emotion(image_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    
    # Resize the image for the model prediction (48x48 grayscale)
    face = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    face_resized = cv2.resize(face, (img_size, img_size))  # Resize the image to 48x48
    face_resized = face_resized.reshape(1, img_size, img_size, 1)
    face_resized = face_resized / 255.0  # Normalize the image

    # Make a prediction
    prediction = model.predict(face_resized)
    max_index = np.argmax(prediction)
    emotion = categories[max_index]
    
    # Print the predicted emotion in the output
    print(f"Predicted Emotion: {emotion}")
    
    # Display the emotion on the image and draw a green frame
    color = (0, 255, 0)  # Green color for the frame and text
    thickness = 2  # Thickness of the frame

    # Draw a green rectangle around the image (simulating a frame)
    h, w, _ = img.shape
    cv2.rectangle(img, (10, 10), (w-10, h-10), color, thickness)

    # Add the predicted emotion text on top of the image
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, emotion, (20, 50), font, 1, color, 2, cv2.LINE_AA)

    # Save the resulting image with the green frame and emotion text
    output_image_path = image_path.replace('.jpg', '_emotion.jpg')  # Modify the filename
    cv2.imwrite(output_image_path, img)
    print(f"Saved the image with emotion: {output_image_path}")

    # Display the resulting image
    cv2.imshow("Emotion Prediction", img)
    
    # Wait for a key press and close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage for a new image
image_path = r'C:\Users\Admin\Desktop\Kids\girl-2961959_640.jpg'  # Replace with the actual path of the new image
predict_emotion(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 722ms/step
Predicted Emotion: Angry
Saved the image with emotion: C:\Users\Admin\Desktop\Kids\girl-2961959_640_emotion.jpg
